In [2]:
pip install pandas pyarrow


Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

# 파일 읽기
df = pd.read_parquet(r'C:\Users\ewq95\github\DataScience\scikit-learn\data\open\train\1.회원정보\201807_train_회원정보.parquet')

# 상위 5개 데이터 확인
print(df.head())

     기준년월            ID  남녀구분코드   연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
0  201807  TRAIN_000000       2  40대       D          1             1   
1  201807  TRAIN_000001       1  30대       E          1             1   
2  201807  TRAIN_000002       1  30대       C          1             1   
3  201807  TRAIN_000003       2  40대       D          1             1   
4  201807  TRAIN_000004       2  40대       E          1             1   

   회원여부_이용가능_카드론  소지여부_신용  소지카드수_유효_신용  ...  할인금액_제휴연회비_B0M  청구금액_기본연회비_B0M  \
0              0        1            1  ...               0               0   
1              1        1            1  ...               0               0   
2              0        1            1  ...               0               0   
3              0        1            2  ...               0               0   
4              1        1            1  ...               0               0   

   청구금액_제휴연회비_B0M  상품관련면제카드수_B0M  임직원면제카드수_B0M  우수회원면제카드수_B0M  기타면제카드수_B0M  \
0       

In [ ]:
import pandas as pd
import os

# 1. 기본 경로 설정
HOME = os.getcwd()
train_path = os.path.join(HOME, 'train')

# 2. 폴더 목록 (이미지에 있는 8개 폴더 이름)
folders = [
    '1.회원정보', '2.신용정보', '3.승인매출정보', '4.청구입금정보',
    '5.잔액정보', '6.채널정보', '7.마케팅정보', '8.성과정보'
]

# 각 속성별(폴더별)로 합쳐진 데이터프레임을 담을 리스트
category_dfs = []

print("데이터 통합 시작!")

for folder in folders:
    folder_path = os.path.join(train_path, folder)
    
    # 해당 폴더 안에 있는 6개의 월별 파일 리스트업
    # 예: 201807_train_회원정보.parquet ~ 201812_train_회원정보.parquet
    monthly_files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]
    monthly_files.sort()  # 월 순서대로 정렬
    
    # (1) 월별 데이터 세로로 합치기 (행 병합)
    temp_monthly_list = []
    for file in monthly_files:
        file_full_path = os.path.join(folder_path, file)
        df_month = pd.read_parquet(file_full_path)
        temp_monthly_list.append(df_month)
    
    # 7~12월 데이터가 하나로 합쳐짐
    combined_category_df = pd.concat(temp_monthly_list, axis=0, ignore_index=True)
    category_dfs.append(combined_category_df)
    
    print(f"✅ {folder} 통합 완료: {combined_category_df.shape}")

# (2) 8개의 속성 데이터를 가로로 합치기 (열 병합)
# 만약 모든 파일의 행 순서와 ID가 완벽히 일치한다면 concat(axis=1)을 쓰고,
# 안전하게 합치려면 공통된 기준 컬럼(예: '고객번호' 등)을 기준으로 merge 해야 해.

# 여기서는 모든 데이터의 행 순서가 동일하다고 가정하고 concat을 먼저 써볼게.
# (만약 기준 ID가 따로 있다면 merge 방식으로 수정해야 하니 말해줘!)
final_df = pd.concat(category_dfs, axis=1)

# 중복된 컬럼(합치면서 중복된 ID 등)이 생겼다면 제거
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

print("-" * 30)
print(f"최종 통합 완료! 데이터 크기: {final_df.shape}")

# 3. 하나로 통합된 파일 저장 (나중에 쓰기 편하게!)
final_df.to_parquet('final_integrated_train_data.parquet')
print("파일 저장 완료: final_integrated_train_data.parquet")

데이터 통합 시작!
✅ 1.회원정보 통합 완료: (2400000, 78)
✅ 2.신용정보 통합 완료: (2400000, 42)
✅ 3.승인매출정보 통합 완료: (2400000, 406)
